_#TODO: Проверь у Клима и Вовы __R__ (разрешающую способность)_

# Лабораторная работа №4.4.1 "Амплитудная дифракционная решетка"

В работе предстоит познакомиться с работой гониометра и исследовать спектральные характеристики спектральных приборов (амплитудной решетки).

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.linear_model import LinearRegression
from astropy.coordinates import Angle # Для работы с углами, минутами и секундами. Установите библиотеку, если что

## 1. Определение шага решетки

Считаем данные:

In [38]:
df = pd.read_csv('1.csv')

In [39]:
df

,m,line,alpha
0,1,6,168°22′19″
1,1,5,167°27′56″
2,1,4,165°50′15″
3,1,3,164°13′41″
4,1,2,163°19′10″
5,1,1,163°15′39″
6,-1,6,191°40′32″
7,-1,5,192°38′38″
8,-1,4,194°17′32″
9,-1,3,195°55′24″


In [40]:
# Переведем строки в углы. Это пригодится в будущем, но испортит вид датафрейма при выводе на экран
# Все углы будут в градусах. Но numpy это поймет, так что переводить не нужно
df['alpha'] = df['alpha'].map(lambda x: Angle(x))
# Не забудем, что у гониометра есть alpha_0. Вычтем его
alpha_0 = Angle('180d1m51s')
df['alpha'] -= alpha_0

В лабнике есть таблица соотвествия: длина волны $\longleftrightarrow$ номер линии. Возьмем ее:

In [41]:
line_map = {1: 579.1, 2: 577.0, 3: 546.1, 
            4: 491.6, 5: 435.8, 6: 404.7}

In [42]:
df['lambda'] = [line_map[x] for x in df['line'].astype(int)]
df.head()

,m,line,alpha,lambda
0,1,6,-11d39m32s,404.7
1,1,5,-12d33m55s,435.8
2,1,4,-14d11m36s,491.6
3,1,3,-15d48m10s,546.1
4,1,2,-16d42m41s,577.0


Построим график $\sin \phi_m$ от длины волны $\lambda$. Ожидаемая зависимость: $\sin \phi_m = \cfrac{m}{d} \lambda$

Для $\underline{m = 1}$ имеем:

In [43]:
reg = LinearRegression(n_jobs=-1)

# m = 1
sub_df = df[df.m == 1]
y_train, x_train = sub_df['alpha'].map(np.sin), sub_df['lambda']
reg.fit(x_train.values.reshape(-1, 1), y_train.values)
k1 = reg.coef_[0]
x_line = np.linspace(0, 1.1*max(x_train), 1000)
y_line = reg.predict(x_line.reshape(-1, 1))

In [44]:
# m = -1
sub_df = df[df.m == -1]
y_train1, x_train1 = sub_df['alpha'].map(np.sin), sub_df['lambda']
reg.fit(x_train1.values.reshape(-1, 1), y_train1.values)
k2 = reg.coef_[0]
x_line1 = np.linspace(0, 1.1*max(x_train1), 1000)
y_line1 = reg.predict(x_line1.reshape(-1, 1))

In [45]:
fig, ax = plt.subplots()
# P.S. У нас что-то пошло не так и для m = 1 синус отрицательный. Поэтому я его свапну с m = -1
ax.scatter(x_train, y_train, marker='v', label=r'Эксперимент, $m = -1$')
ax.scatter(x_train1, y_train1, marker='^', label=r'Эксперимент, $m = 1$')
ax.plot(x_line, y_line, label=r'Лин. регрессия, $m = -1$')
ax.plot(x_line1, y_line1, label=r'Лин. регрессия, $m = 1$')
# Оси
ax.set_xlim((min(min(x_line), min(x_line1)), max(max(x_line1), max(x_line))))
ax.xaxis.set_major_locator(MultipleLocator(100))
ax.xaxis.set_minor_locator(MultipleLocator(25))
ax.yaxis.set_major_locator(MultipleLocator(0.1))
ax.yaxis.set_minor_locator(MultipleLocator(0.025))

# Подписи
ax.set_xlabel(r'$\lambda$, нм')
ax.set_ylabel(r'$\sin\,(\phi_m)$')
ax.text(40, .25, r'$k_1 = $ %s, $k_2$ = %s, $\left[\frac{\mathrm{штрих}}{\mathrm{мм}}\right]$'
        % (round(k1*10**6, 2), round(k2*10**6, 2)), 
        bbox={'facecolor':'white', 'alpha': 1, 'pad':10})
ax.set_title(r'Определение шага решетки')
# Прочая красота
ax.grid(which='minor', c='black', linestyle='--', alpha=0.2)
ax.grid(which='major', c='black')
ax.legend()
fig.savefig('1.pdf')

In [48]:
d = (-k1+k2)/2*10**6

In [49]:
print('Итого: {} штрих/мм'.format(d))

Итого: 500.92835425639896 штрих/мм


## 2. Определение угловой дисперсии

In [50]:
df = pd.read_csv('2.csv')

In [51]:
df

,m,line1,line2
0,1,163°15′39″,163°19′10″
1,2,144°54′30″,145°2′50″
2,3,120°51′38″,121°11′40″
3,-1,196°54′46″,196°50′51″
4,-2,215°40′37″,215°31′53″
5,-3,241°34′37″,241°11′49″


In [52]:
# Переведем строки в углы. Это пригодится в будущем, но испортит вид датафрейма при выводе на экран
# Все углы будут в градусах. Но numpy это поймет, так что переводить не нужно
df['line1'] = df['line1'].map(lambda x: Angle(x))
df['line2'] = df['line2'].map(lambda x: Angle(x))
# Не забудем, что у гониометра есть alpha_0. Вычтем его
alpha_0 = Angle('180d1m53s')
df['line1'] -= alpha_0
df['line2'] -= alpha_0

In [53]:
l2 = 577
l1 = 579.1

In [54]:
df['disp'] = (df['line2'] - df['line1'])/(l2 - l1)
# Угловая дисперсия мерится в град/нм !!!
# Опять же, мы мудаки. У нас m=1 спутался с m=-1
df['disp'] *= -1

In [55]:
df

,m,line1,line2,disp
0,1,-16d46m14s,-16d42m43s,0d01m40.4762s
1,2,-35d07m23s,-34d59m03s,0d03m58.0952s
2,3,-59d10m15s,-58d50m13s,0d09m32.381s
3,-1,16d52m53s,16d48m58s,-0d01m51.9048s
4,-2,35d38m44s,35d30m00s,-0d04m09.5238s
5,-3,61d32m44s,61d09m56s,-0d10m51.4286s


Строим график этого безобразия. В качестве $d$ возьмем указанную на установке: $d = 500~\mathrm{штрих}/\mathrm{нм}$ 

In [80]:
df['disp_d'] = df['disp'].map(lambda x: x.to_value())

In [81]:
y_train, x_train = df['disp_d']/180*np.pi, df['m']

x_line = np.linspace(1.1*min(x_train), 1.1*max(x_train), 1000)
y_line = np.apply_along_axis(lambda x: x/np.sqrt((1/d*10**6)**2-x**2*((l1+l2)/2)**2), 0, x_line)

In [117]:
fig, ax = plt.subplots()
# P.S. У нас что-то пошло не так и для m = 1 синус отрицательный. Поэтому я его свапну с m = -1
ax.scatter(x_train, y_train, marker='v', label=r'Эксперимент')
ax.plot(x_line, y_line, label=r'Теоретическая зависимость')
# Оси
#ax.set_xlim(min(x_line), max(x_line))
ax.set_ylim((-0.006, 0.006))
ax.yaxis.set_label_coords(-0.07, 0.5)
ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
#ax.xaxis.set_major_locator(MultipleLocator(100))
#ax.xaxis.set_minor_locator(MultipleLocator(25))
ax.yaxis.set_major_locator(MultipleLocator(1*10**-3))
ax.yaxis.set_minor_locator(MultipleLocator(0.5*10**-3))

# Подписи
ax.set_xlabel(r'$m$')
ax.set_ylabel(r'$\phi_m$, рад')
#ax.text(40, .25, r'$k_1 = $ %s, $k_2$ = %s, $\left[\frac{\mathrm{штрих}}{\mathrm{мм}}\right]$'
#        % (round(k1*10**6, 2), round(k2*10**6, 2)), 
#        bbox={'facecolor':'white', 'alpha': 1, 'pad':10})
ax.set_title(r'Угловая дисперсия')
# Прочая красота
ax.grid(which='minor', c='black', linestyle='--', alpha=0.2)
ax.grid(which='major', c='black')
ax.legend()
fig.savefig('2.pdf')

/usr/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


## 3. Оценка разрешающей способности и числа штрихов

In [154]:
df = pd.read_csv('4.csv')

In [155]:
df

,m,left,right
0,-1,196°54′17″,196°55′05″
1,-2,215°40′15″,215°41′09″
2,-3,241°33′51″,241°36′40″


In [156]:
# Переведем строки в углы. Это пригодится в будущем, но испортит вид датафрейма при выводе на экран
# Все углы будут в градусах. Но numpy это поймет, так что переводить не нужно
df['left'] = df['left'].map(lambda x: Angle(x))
df['right'] = df['right'].map(lambda x: Angle(x))
# Не забудем, что у гониометра есть alpha_0. Вычтем его
alpha_0 = Angle('180d1m51s')
df['left'] -= alpha_0
df['right'] -= alpha_0
df['left_d'] = df['left'].map(lambda x: x.to_value())
df['right_d'] = df['right'].map(lambda x: x.to_value())

In [157]:
df['2delta'] = df['right_d'] - df['left_d']

In [158]:
df['delta'] = df['2delta']/2

По этим данным оценим $\Delta \phi = \cfrac{m}{d \cos \phi_m} \delta \lambda$, а затем уже
$R = \cfrac{\lambda}{\delta \lambda} = \cfrac{\lambda m}{d \Delta \phi \cos \phi_m}$

In [161]:
l = (l1+l2)/2

In [171]:
df['R'] = np.abs(l*df['m']*10**(-6)*d/(df['delta']*np.cos(df['left_d'])))

In [172]:
df['R']

0    110.267209
1    164.591827
2    137.618597
Name: R, dtype: float64

In [180]:
df

,m,left,right,left_d,right_d,2delta,delta,R
0,-1,16d52m26s,16d53m14s,16.873889,16.887222,0.013333,0.006667,110.267209
1,-2,35d38m24s,35d39m18s,35.640000,35.655000,0.015000,0.007500,164.591827
2,-3,61d32m00s,61d34m49s,61.533333,61.580278,0.046944,0.023472,137.618597


In [181]:
df['N'] = df['R']/df['m']

In [182]:
df

,m,left,right,left_d,right_d,2delta,delta,R,N
0,-1,16d52m26s,16d53m14s,16.873889,16.887222,0.013333,0.006667,110.267209,-110.267209
1,-2,35d38m24s,35d39m18s,35.640000,35.655000,0.015000,0.007500,164.591827,-82.295913
2,-3,61d32m00s,61d34m49s,61.533333,61.580278,0.046944,0.023472,137.618597,-45.872866


In [173]:
R = np.mean(df['R'])

In [179]:
print('R = {}'.format(int(round(R))))

R = 137
